# First Implementation Of Gemma in PathVQA


In [ ]:
from huggingface_hub import login
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from pathlib import Path
from datasets import load_from_disk
from PIL import Image
import requests
import torch


In [ ]:
model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

In [16]:
project_root = Path.cwd().parent
data_path = project_root / "data"  / "train"
 
dataset = load_from_disk(str(data_path))
sample = dataset[1]  
image = sample["image"] 
question = sample["question"]
answer = sample["answer"]

In [19]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": question}
        ]
    }
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

# **Overall Impression:** The image is a close-up shot of a vibrant garden scene, 
# focusing on a cluster of pink cosmos flowers and a busy bumblebee. 
# It has a slightly soft, natural feel, likely captured in daylight.


C:\Users\lukas\AppData\Roaming\Python\Python311\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\lukas\AppData\Roaming\Python\Python311\site-packages\transformers\generation\configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Okay, let's analyze the image and determine what's being stained with an immunohistochemical stain for cytokeratin 7.

**Based on the image, the stained structures are likely *tumor cells*.**

Here's the reasoning:

*   **Brown Staining:** The brown staining is characteristic of antibody-antigen binding. In this case, the antibody specifically recognizes cytokeratin 7 (CK7), a protein commonly found in epithelial cells.
*   **
